In [1]:
import cv2
import numpy as np

# Open a simple image
img=cv2.imread("h1.jpg")

In [2]:
def detect_hand_region(img, show=False):
    # converting from gbr to hsv color space
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # skin color range for hsv color space
    HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255))
    HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    # converting from gbr to YCbCr color space
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    # skin color range for hsv color space
    YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135))
    YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    # merge skin detection (YCbCr and hsv)
    global_mask = cv2.bitwise_and(YCrCb_mask,HSV_mask)
    global_mask = cv2.medianBlur(global_mask,3)
    global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4,4), np.uint8))

    #show results
    if show:
        cv2.imshow("1_HSV",HSV_mask)
        cv2.imshow("2_YCbCr",YCrCb_mask)
        cv2.imshow("3_global_result",global_mask)
        cv2.imshow("org",img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return global_mask

In [3]:
def is_contour_close_to_border(contour, img_shape, threshold):
    x = contour[:,:,0]
    y = contour[:,:,1]
    # left, right = min(x), max(x)
    # top, bottom = min(y), max(y)
    bottom = max(y)
    height, width = img_shape[:2]
    
    if height - bottom < threshold:
        return True
    return False


def identify_hand_contour(binary, show=False):
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=lambda x: len(x), reverse=True)
    
    hand_contour = None
    for cnt in contours:
        if is_contour_close_to_border(cnt, img.shape, img.shape[0]*0.1):
            hand_contour = cnt
            break
    
    if hand_contour is not None:
        if show:
            draw_cnt = np.zeros((binary.shape[0], binary.shape[1], 3), dtype=np.uint8)
            cv2.drawContours(draw_cnt, [hand_contour], 0, (255,255,0), 2)
            cv2.imshow("hand contour",draw_cnt)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
    else:
        print('No hand region found')
    return hand_contour

In [4]:
def detect_fingertip(contour, show=False, image=None):
    fingertip = min(contour, key=lambda x: x[0][1])       
    fingertip = np.reshape(fingertip, (2))
    
    if show and image is not None:
        board = image.copy()
        cv2.circle(board, fingertip, 5, (0,255,0), 2)
        cv2.imshow('fingertip', board)
        cv2.waitKey()
        cv2.destroyAllWindows()
    return fingertip

In [5]:
binary = detect_hand_region(img)
hand_contour = identify_hand_contour(binary)
detect_fingertip(hand_contour, image=img)

array([252,  52], dtype=int32)

In [7]:
# *******************************
# *** Adjust Video Clip Range ***
# *******************************

import cv2

def nothing(x):
    pass


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    
    # 1. Color-range-based hand region detection and binarization
    binary = detect_hand_region(frame)
    cv2.imshow("binary", binary)
    
    # 2. Identify hand contour
    hand_contour = identify_hand_contour(binary)
    if hand_contour is not None:
        draw_cnt = np.zeros((binary.shape[0], binary.shape[1], 3), dtype=np.uint8)
        cv2.drawContours(draw_cnt, [hand_contour], 0, (255,255,0), 2)
        cv2.imshow("hand contour",draw_cnt)
    
    # 3. Detect the fingertip
    if hand_contour is not None:
        fingertip = detect_fingertip(hand_contour)
        draw_fingertip = frame.copy()
        cv2.circle(draw_fingertip, fingertip, 5, (0,255,0), 2)
        cv2.imshow('fingertip', draw_fingertip)

        
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
